# Interactive Clean
The CASA interactive clean application is available for use in Jupyter notebooks. The primary difference between usage in Jupyter and usage directly from a Python command line interface (CLI) is that the `iclean.notebook` function is used within a notebook instead of the `iclean` function that is used from the Python CLI.

## Current status

Currently the interactive clean app works well when a **single** instance is run within a notebook **and** the Jupyter kernel is running on the same host as the browser displaying the notebook. Things begin to go wrong when the *red stop sign* is pressed. Doing this is necessary because this is how the `iclean` return dictionary is returned. The `get_result` function blocks until the *red stop sign* is clicked and confirmed. At this point, the `iclean` return dictionary is returned as the result of `get_result`. Another function called `get_future` is available which does not block but instead retrieved the future where the `iclean` return dictionary will eventually appear.

Supporting, multiple `iclean` instances within a single worksheet is the goal, but currently only one instance should be used, *unless you are willing to risk corruption of your Jupyter session*. All of the GUI display cells in this notebook are beleved to work when a **single** one is executed within a given notebook instance.

## Which version of cubevis to use

The current version of `cubevis` to use is [v1.0.14](https://pypi.org/project/cubevis/1.0.14/).

## Setup
This first cell ensures that CASA 6 and cubevis are available:

In [ ]:
!mkdir -p ~/.casa/data
!pip install --upgrade "casatasks>=6.7.2.42" cubevis

This cell simply retrieves the test data that is required.

In [ ]:
import os
#os.environ['CUBEVIS_JS_TAG']='jupyter'
import ssl
import certifi
import urllib
import tarfile
ms_path = 'refim_point_withline.ms'
ms_url = "https://casa.nrao.edu/download/devel/casavis/data/refim_point_withline-ms.tar.gz"
if not os.path.isdir(ms_path):
    try:
        context = ssl.create_default_context(cafile=certifi.where())
        tstream = urllib.request.urlopen(ms_url, context=context, timeout=400)
        tar = tarfile.open(fileobj=tstream, mode="r:gz")
        tar.extractall(filter='data')
    except urllib.error.URLError:
        print("Failed to open connection to "+ms_url)

## Import
The first step is to import the `iclean` function:

In [ ]:
from cubevis import iclean

## Cleanup
`iclean` operates on state contained within files in the current directory, so it is always a good idea to make sure that you are starting with a clean slate. This command removes the state files that accumulate from running this example.

In [ ]:
!rm -rf test.*

## Create the Application Object
This cell creates the interactive clean application object. This object is used to display the GUI and to retreive the `iclean` return dictionary from the GUI once the user has completed cleaning.

In [ ]:
ic = iclean.notebook( vis=ms_path, imagename='test',
                      imsize=512, niter=50,
                      cycleniter=10,
                      cell='12.0arcsec',
                      specmode='cube', interpolation='nearest',
                      nchan=5, start='1.0GHz', width='0.2GHz', pblimit=-1e-05,
                      deconvolver='hogbom', threshold='0.001Jy', cyclefactor=3, scales=[0,3,10] )

## Display the GUI
Once the application object has been created, the GUI can be displayed:

In [ ]:
ic.show( )

## Retrieve the result
Once the user has completed cleaning, clicked the stop sign and approved the exiting the application, the result can be retrieved:

In [ ]:
ic.get_result( )

## Alternate ways to display the GUI
While this is the most obvious way to create and use the interactive clean within a Jupyter notebook, there are also some subtle variations.

### Display as a result of cell evaulation
The last evaluated statement in a notebook cell is displayed and displaying the `iclean` application object results in the GUI being displayed.

In [ ]:
!rm -rf test.*
iclean.notebook( vis=ms_path, imagename='test',
                 imsize=512, niter=50,
                 cycleniter=10,
                 cell='12.0arcsec',
                 specmode='cube', interpolation='nearest',
                 nchan=5, start='1.0GHz', width='0.2GHz', pblimit=-1e-05,
                 deconvolver='hogbom', threshold='0.001Jy', cyclefactor=3, scales=[0,3,10] )

### Retrieving the result
Even though we did not retain the application object, the value of the **last** cell evaluation is available as `_`. So using this, we can retrieve the return dictionary:

In [ ]:
_.get_result( )

### Using Bokeh's show function
One last slight variation is to use [Bokeh's](https://bokeh.org/) `show` function to display the application object:

In [ ]:
!rm -rf test.*
from bokeh.io import output_notebook
from bokeh.plotting import show
output_notebook( )
ic = iclean.notebook( vis=ms_path, imagename='test',
                      imsize=512, niter=50,
                      cycleniter=10,
                      cell='12.0arcsec',
                      specmode='cube', interpolation='nearest',
                      nchan=5, start='1.0GHz', width='0.2GHz', pblimit=-1e-05,
                      deconvolver='hogbom', threshold='0.001Jy', cyclefactor=3, scales=[0,3,10] )
show(ic)